# Digital-Contact-Tracing on DTU

This notebook demonstrates the use of the simulator on the DTU dataset.

### Imports and setup

In [1]:
import DigitalContactTracing as DCT
from load_temporal_graph import load_df, build_graphs, get_array_of_contacts

%load_ext autoreload
%autoreload 2

### Define the parameters of the simulation

In [2]:
PARAMETERS = {"temporal_gap" : 900,                 # Temporal gap between static networks (sec)
              "memory_contacts" : 7,                # Tracing memory (days)
              "max_time_quar" : 10,                 # Quarantine duration (days)
              "policies" : {"policy1":[-100,600]},  # Digital tracing policies [RSSI, duration (sec)]
              "eps_Is" : [0.5,0.2],                 # Isolation effectivity
              "times" : 2,                          # Number of repetition of the simulation
              "seed" : 11,                          # Random seed 
              "Y_i" : 10,                           # Number of initially infected people
              "nc" : 0.2,                           # Fraction of individuals who are not adopting the app
              "beta_t": 0.002,                      # Parameter defining the infectiousness probability
              "symptomatics" : 0.8,                 # Fraction of symptomatic individuals 
              "testing" : 0.25,                     # Fraction of asymptomatics who are detected via random testing
              "store":{"to_store": True,            # Save the results?
                       "path_to_store":"RESULTS/"}} # Target folder to save the results

### Load the temporal graph

In [3]:
df = load_df("Dataset//bt_symmetric.csv", n_row =100000, n_individuals=100)
graphs = build_graphs(get_array_of_contacts(df, PARAMETERS["temporal_gap"], column_name='# timestamp'), 
                      PARAMETERS["temporal_gap"])

### Run the simulations and store the results

In [4]:
# Initialize the random seed
np.random.seed(PARAMETERS["seed"])

# Run over each configuration
for eps_I in PARAMETERS["eps_Is"]:
    for policy in PARAMETERS["policies"]:
        filter_rssi = PARAMETERS["policies"][policy][0]
        filter_duration = PARAMETERS["policies"][policy][1]
        results = []
        
        # Print some report
        print("-" * 100)
        print('Running simulation with policy: %s' % policy)
        print('       |_ eps_I             : %+4.2f'   % eps_I)
        print('       |_ filter_rssi       : %+4.2f' % filter_rssi)
        print('       |_ filter_duration   : %+4.2f' % filter_duration)
        print('')
        
        # Repeat the simulation for a fixed configuration 
        for i in range(PARAMETERS["times"]):
            # Print some report
            print('       Running simulation     [%4d / %4d]' % (i + 1, PARAMETERS['times']))

            # Initialize the simulation object
            dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration)
            # Run the actual simulation
            res = dct.simulate()
            
            # Save the results    
            if PARAMETERS["store"]["to_store"]:    
                DCT.store_real_time(res,PARAMETERS,filter_rssi,filter_duration,eps_I)

----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I             : +0.50
       |_ filter_rssi       : -100.00
       |_ filter_duration   : +600.00

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]
----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I             : +0.20
       |_ filter_rssi       : -100.00
       |_ filter_duration   : +600.00

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]
